In [3]:
## Starting by downloading documents from gmail as named to a folder 

import imaplib
import os
import email
import sys
import json
import tkinter as tk
from tkinter import filedialog

class DocumentExtractor():
    
    def __init__(self):
        self.username = ""
        self.password = ""
        self.mail = object
        self.mailbox = "Inbox"
        self.mailCount = 0
        self.docFolder = ""
        self.data = []
        self.ids = []
        self.idsList = []
        
    def get_login(self):
        self.username = input("Enter your gmail address: ")
        self.password = input("Enter your password: ")
        
    def attempt_login(self):
        self.mail = imaplib.IMAP4_SSL("imap.gmail.com", 993)
        if self.mail.login(self.username, self.password):
            print("Login successful!")
            ## Using TKinter to get the folder to save the documents without user having to type in the path
            root = tk.Tk()
            root.withdraw()
            self.docFolder = filedialog.askdirectory(mustexist=True)
            return True
        else:
            print("Login failed. Please try again.")
            return False
        
    def get_mailbox(self):
        inbox_count = self.mail.select(self.mailbox)[1]
        self.mailCount = int(inbox_count[0].decode("utf-8"))
        print("You have {} messages in your inbox.".format(self.mailCount))
        if self.mailCount == 0:
            new_mailbox = input("Would you like to choose another mailbox to download from? Y/N:")
            if new_mailbox == "Y" or "y":
                self.mailbox = input("Enter the name of the mailbox you would like to download from: ")
                self.get_mailbox()
            else:
                print("No messages to download. Exiting.")
                sys.exit()
        return True if self.mailCount > 0 else False
    
    def search_mailbox(self): 
        type, self.data = self.mail.search(None, "ALL")
        self.ids = self.data[0]
        self.idsList = self.ids.split()
        
    def download_documents(self):
        for message in self.data[0].split():
            type, self.data = self.mail.fetch(message, '(UID RFC822)')
            raw = self.data[0][1]
            try:
                raw_str = raw.decode("utf-8")
            except UnicodeDecodeError:
                try:
                    raw_str = raw.decode("ISO-8859-1") 
                except UnicodeDecodeError:
                    try:
                        raw_str = raw.decode("ascii") 
                    except UnicodeDecodeError:
                        pass
						
            msg = email.message_from_string(raw_str)
            
            if msg.is_multipart():
                for part in msg.walk(): 
                    if part.get('Content-Disposition') is None: 
                        ## this get filename will change after implementing the model 
                        attchName = part.get_filename()
                        if bool(attchName):
                            attchFilePath = str(self.destFolder)+str(uid)+str("/")+str(attchName)
                            os.makedirs(os.path.dirname(attchFilePath), exist_ok=True)
                            with open(attchFilePath, "wb") as f:
                                f.write(part.get_payload(decode=True))
            else:
                ## no attachment, nothing to download
                pass
            
    def main(self):
        self.get_login()
        if self.attempt_login():
            if self.get_mailbox():
                self.search_mailbox()
                self.download_documents()
            else:
                print("No messages to download. Exiting.")
                sys.exit()
        else:
            print("Login failed. Exiting.")
            sys.exit()
            
DocumentExtractor().main()

error: b'[AUTHENTICATIONFAILED] Invalid credentials (Failure)'